In [1]:
from db_queries import get_population, get_ids
from db_queries import get_location_metadata as get_locs

ModuleNotFoundError: No module named 'db_queries'

In [ ]:
import pandas as pd, numpy as np

# LSFF: choose population coverage data by hand for tier 5 countries

## vehicles: Wheat flour, maize flour, oil

## countries: 	Angola, China, Ghana, Niger, Egypt, Sudan, Madagascar

In [ ]:
## load legal combos
import pickle
data_prep_dir = '/ihme/homes/beatrixh/vivarium_research_lsff/data_prep/inputs/'

with open(data_prep_dir + 'lsff_vehicle_nutrient_pairs.pickle', 'rb') as handle:
    vehicle_nutrient_map = pickle.load(handle)
    
with open(data_prep_dir + 'lsff_country_vehicle_pairs.pickle', 'rb') as handle:
    country_vehicle_map = pickle.load(handle)
    
with open(data_prep_dir + 'lsff_vehicle_country_pairs.pickle', 'rb') as handle:
    vehicle_country_map = pickle.load(handle)

In [ ]:
nutrients = ['iron','zinc','folic acid','vitamin a']

In [ ]:
data_path = '/ihme/homes/beatrixh/vivarium_research_lsff/data_prep/inputs/extraction_sheet_lsff_03_24_2021.3.csv'
assm_path = '/ihme/homes/beatrixh/vivarium_research_lsff/data_prep/inputs/extraction_sheet_lsff_assumed_coverage_03_24_2021.csv'

df = pd.read_csv(data_path)

In [ ]:
df.loc[df.nutrient=="folic acid, folate, b9",'nutrient']= 'folic acid'

In [ ]:
assum = pd.read_csv(assm_path)

In [ ]:
#these don't apply this time
assum.location_name.unique()

In [ ]:
assum.loc[assum.nutrient=="folic acid, folate, b9",'nutrient']= 'folic acid'

In [ ]:
df['estimation_status'] = 'na'

In [ ]:
df['data_choice_notes'] = ""

In [ ]:
mult_estimates_path = '/ihme/homes/beatrixh/vivarium_research_lsff/data_prep/outputs/lsff_data_estimated_03_26_2021.4.csv'

mult_estimates = pd.read_csv(mult_estimates_path)

In [ ]:
#reformat
mult_estimates.loc[(mult_estimates.B_estimate!=1.0),'B'] = np.nan
mult_estimates.loc[(mult_estimates.C_estimate!=1.0),'C'] = np.nan

mult_estimates = pd.melt(mult_estimates,
                         id_vars = ['location_name','vehicle','nutrient','standard'],
                         value_vars = ['B','C'], var_name = 'value_description', value_name = 'value_mean').dropna()

mult_estimates.loc[(mult_estimates.value_description=="B"),'nutrient'] = 'NA'

mult_estimates = mult_estimates.drop_duplicates()

mult_estimates.value_description = mult_estimates.value_description.map({
    'B':'percent of population eating industrially produced vehicle',
    'C':'percent of population eating fortified vehicle'
})

In [ ]:
mult_estimates['estimation_status'] = 'multiplicative'

In [ ]:
def prep_reg_estimates(path):
    draws = [f'draw_{i}' for i in range(500)]

    df = pd.read_csv(path)
    df = df.groupby(['location_name','vehicle']).mean().reset_index()
    df['value_mean'] = df[draws].mean(axis=1)
    df['value_025_percentile'] = df[draws].quantile(.025, axis=1)
    df['value_975_percentile'] = df[draws].quantile(.975, axis=1)

    return df[['location_name','vehicle','value_mean']]

In [ ]:
output_dir = '/ihme/homes/beatrixh/vivarium_research_lsff/data_prep/outputs/'

reg_fort_oil_path = output_dir + 'pct_eating_fortified_oil_regression_estimates_3_22_2021.csv'
reg_fort_wheat_path = output_dir + 'pct_eating_fortified_wheat_regression_estimates_3_22_2021.csv'
reg_fort_maize_path = output_dir + 'pct_eating_fortified_maize_regression_estimates_3_22_2021.csv'

est_fortified = pd.concat([prep_reg_estimates(path) for path in [reg_fort_oil_path,reg_fort_wheat_path,reg_fort_maize_path]])
est_fortified['value_description'] = "percent of population eating fortified vehicle"
est_fortified['estimation_status'] = "regression"

In [ ]:
vn_pairs = pd.DataFrame([(v,n) for v in ['oil','wheat flour','maize flour'] for n in vehicle_nutrient_map[v]],
            columns=['vehicle','nutrient'])

In [ ]:
est_fortified = est_fortified.merge(vn_pairs, on = 'vehicle', how = 'outer')[['location_name','vehicle','value_description','nutrient','value_mean','estimation_status']]

In [ ]:
ind_prod_oil_path = '/ihme/homes/beatrixh/vivarium_research_lsff/data_prep/outputs/pct_eating_fortifiable_oil_regression_estimates_3_26_2021.csv'
ind_prod_wheat_path = '/ihme/homes/beatrixh/vivarium_research_lsff/data_prep/outputs/pct_eating_fortifiable_wheat_regression_estimates_3_26_2021.csv'
ind_prod_maize_path = '/ihme/homes/beatrixh/vivarium_research_lsff/data_prep/outputs/pct_eating_fortifiable_maize_regression_estimates_3_26_2021.csv'

est_fortifiable = pd.concat([prep_reg_estimates(path) for path in [ind_prod_oil_path,ind_prod_wheat_path,ind_prod_maize_path]])
est_fortifiable['value_description'] = "percent of population eating industrially produced vehicle"
est_fortifiable['estimation_status'] = "regression"
est_fortifiable['nutrient'] = "na"

In [ ]:
eating_oil_path = output_dir + 'pct_eating_oil_regression_estimates_3_23_2021.csv'
eating_wheat_path = output_dir + 'pct_eating_wheat_regression_estimates_3_22_2021.csv'
eating_maize_path = output_dir + 'pct_eating_maize_regression_estimates_3_22_2021.csv'

est_eating = pd.concat([prep_reg_estimates(path) for path in [eating_oil_path,eating_wheat_path,eating_maize_path]])
est_eating['value_description'] = "percent of population eating vehicle"
est_eating['estimation_status'] = "regression"
est_eating['nutrient'] = "na"

In [ ]:
reg_estimates = pd.concat([est_eating, est_fortifiable, est_fortified])

In [ ]:
location_names = ['Angola', 'China', 'Ghana', 'Niger', 'Egypt', 'Sudan', 'Madagascar']
vehicles = ['maize flour','wheat flour','oil']
nutrients = ['folic acid','iron','zinc','vitamin a']

In [ ]:
# these are the vehicles per country we need
target_high_level = pd.DataFrame([(loc,v) for loc in location_names for v in country_vehicle_map[loc]],
            columns=['location_name','vehicle']).sort_values(['location_name','vehicle'])

target_high_level = target_high_level[target_high_level.vehicle.isin(vehicles)].set_index(['location_name','vehicle'])

target_high_level

In [ ]:
target_a = pd.DataFrame([(loc,vehicle,nutrient,'percent of population eating fortified vehicle') for loc in location_names
                       for vehicle in country_vehicle_map[loc]
                      for nutrient in vehicle_nutrient_map[vehicle]],
            columns=['location_name','vehicle','nutrient','value_description']).sort_values(['location_name','vehicle','nutrient'])
target_a = target_a[(target_a.nutrient.isin(nutrients))]

target_b = pd.DataFrame([(loc,vehicle,'na',val) for loc in location_names
                       for vehicle in country_vehicle_map[loc]
                      for val in ['percent of population eating industrially produced vehicle',
       'percent of population eating vehicle']],
            columns=['location_name','vehicle','nutrient','value_description'])

In [ ]:
sortvars = ['location_name','vehicle','value_description','nutrient']
target = target_a.append(target_b)
target = target[(target.vehicle.isin(vehicles)) & (target.nutrient.isin(nutrients + ['na']))].sort_values(sortvars).set_index(sortvars)

In [ ]:
target = target.reset_index()
rcols = target.columns.tolist()

In [ ]:
check_cols = ['location_id','location_name','urbanicity','subnational_name','vehicle','value_description','nutrient','value_mean','value_025_percentile',
       'value_975_percentile','sub_population','source_year','notes','source_citation','source_link','inclusion_justification','included','data_choice_notes']

def filter_data(country, vehicle, val):    
    output = df.loc[(df.location_name==country)
           & (df.vehicle==vehicle)
           & (df.value_description==val)
           & (df.value_mean.notna()),check_cols]
    
    return output

In [ ]:
def check_one_country(country):
    vehicles = ['oil', 'wheat flour', 'salt', 'maize flour', 'rice', 'bouillon']
    values_gold = ['percent of population eating fortified vehicle',
               'percent of population eating industrially produced vehicle',
               'percent of population eating vehicle']
    return pd.concat([filter_data(country, vehicle, val) for vehicle in vehicles for val in values_gold])

In [ ]:
usecols = ['location_id','location_name','subnational_name','vehicle','value_description','nutrient','value_mean', 'value_025_percentile',
       'value_975_percentile']
subset_data = {}

In [ ]:
for i in location_names:
    subset_data[i] = pd.DataFrame()

In [ ]:
location_names

## Angola

In [ ]:
angola = check_one_country("Angola")

angola[(angola.vehicle.isin(vehicles))].groupby(['vehicle','value_description']).mean()

In [ ]:
mult_estimates[(mult_estimates.location_name=="Angola")]

In [ ]:
# wow, yikes

## China

In [ ]:
china = check_one_country("China")

china[(china.vehicle.isin(vehicles))].groupby(['vehicle','value_description']).mean()

In [ ]:
mult_estimates[(mult_estimates.location_name=="China")]

In [ ]:
# oh gosh...

## Ghana

In [ ]:
ghana = check_one_country("Ghana")

ghana[(ghana.vehicle.isin(vehicles))].groupby(['vehicle','value_description']).mean()

In [ ]:
ghana.loc[(ghana.value_description=="percent of population eating fortified vehicle")]

In [ ]:
ghana.loc[(ghana.urbanicity=="mixed/both") & (ghana.value_description=="percent of population eating fortified vehicle"),
         "data_choice_notes"] = "Only one source; discarded urban and rural specific for nationally representative"

subset_data['Ghana'] = subset_data['Ghana'].append(
    ghana.loc[(ghana.urbanicity=="mixed/both") & (ghana.value_description=="percent of population eating fortified vehicle")]
)

ghana.loc[(ghana.urbanicity=="mixed/both") & (ghana.value_description=="percent of population eating fortified vehicle")]

In [ ]:
ghana.loc[(ghana.value_description=="percent of population eating vehicle") & (ghana.vehicle=="oil")]


In [ ]:
ghana.loc[(ghana.urbanicity=="mixed/both") & (ghana.source_year=="2014") & (ghana.value_description=="percent of population eating vehicle") & (ghana.vehicle=="oil"),
         "data_choice_notes"] = "Discarded 2004 Nyumuah number (98%) in lieu of 2014 DHS number (95.6%)"

subset_data['Ghana'] = subset_data['Ghana'].append(
    ghana.loc[(ghana.urbanicity=="mixed/both") & (ghana.source_year=="2014") & (ghana.value_description=="percent of population eating vehicle") & (ghana.vehicle=="oil")]
)

ghana.loc[(ghana.urbanicity=="mixed/both") & (ghana.source_year=="2014") & (ghana.value_description=="percent of population eating vehicle") & (ghana.vehicle=="oil")]

In [ ]:
mult_estimates[(mult_estimates.location_name=="Ghana")]

In [ ]:
subset_data['Ghana'] = subset_data['Ghana'].append(
    mult_estimates[(mult_estimates.location_name=="Ghana")]
)

## Niger

In [ ]:
niger = check_one_country("Niger")

niger[(niger.vehicle.isin(vehicles))].groupby(['vehicle','value_description']).mean()

In [ ]:
niger[(niger.vehicle=="oil") & (niger.value_description=="percent of population eating fortified vehicle")]

In [ ]:
niger.loc[(niger.source_year=="2013") & (niger.vehicle=="oil") & (niger.value_description=="percent of population eating fortified vehicle"),
         "data_choice_notes"] = "From two sources, we had an estimate of 55% in 2010. The latter source additionally projected 75% by 2013, which we are using here."

subset_data['Niger'] = subset_data['Niger'].append(
    niger.loc[(niger.source_year=="2013") & (niger.vehicle=="oil") & (niger.value_description=="percent of population eating fortified vehicle")]
)

niger.loc[(niger.source_year=="2013") & (niger.vehicle=="oil") & (niger.value_description=="percent of population eating fortified vehicle")]

In [ ]:
niger[(niger.vehicle=="oil") & (niger.value_description=="percent of population eating vehicle")]

In [ ]:
niger.loc[(niger.urbanicity=="mixed/both") & (niger.vehicle=="oil") & (niger.value_description=="percent of population eating vehicle"),
         "data_choice_notes"] = "Only one source. Discarded urban and rural specific estimates in lieu of the total estimate"

subset_data['Niger'] = subset_data['Niger'].append(
    niger.loc[(niger.urbanicity=="mixed/both") & (niger.vehicle=="oil") & (niger.value_description=="percent of population eating vehicle")]
)

niger.loc[(niger.urbanicity=="mixed/both") & (niger.vehicle=="oil") & (niger.value_description=="percent of population eating vehicle")]

In [ ]:
niger[(niger.vehicle=="wheat flour") & (niger.value_description=="percent of population eating fortified vehicle")]

In [ ]:
niger.loc[(niger.vehicle=="wheat flour") & (niger.value_description=="percent of population eating fortified vehicle"),
         "data_choice_notes"] = "Only one source."

subset_data['Niger'] = subset_data['Niger'].append(
    niger[(niger.vehicle=="wheat flour") & (niger.value_description=="percent of population eating fortified vehicle")]
)

niger[(niger.vehicle=="wheat flour") & (niger.value_description=="percent of population eating fortified vehicle")]

In [ ]:
mult_estimates[(mult_estimates.location_name=="Niger") 
               & (mult_estimates.vehicle.isin(vehicles)) 
               & (mult_estimates.nutrient.isin(nutrients))]

## Egypt

In [ ]:
egypt = check_one_country("Egypt")

egypt[(egypt.vehicle.isin(vehicles))].groupby(['vehicle','value_description']).mean()

In [ ]:
mult_estimates[(mult_estimates.location_name=="Egypt") 
               & (mult_estimates.vehicle.isin(vehicles)) 
               & (mult_estimates.nutrient.isin(nutrients))]

## Sudan

In [ ]:
sudan = check_one_country("Sudan")

sudan[(sudan.vehicle.isin(vehicles))].groupby(['vehicle','value_description']).mean()

In [ ]:
sudan.loc[(sudan.value_description=="percent of population eating fortified vehicle"),
         "data_choice_notes"] = "Only one source."

subset_data['Sudan'] = subset_data['Sudan'].append(
    sudan.loc[(sudan.value_description=="percent of population eating fortified vehicle")]
)

sudan.loc[(sudan.value_description=="percent of population eating fortified vehicle")]

In [ ]:
sudan.loc[(sudan.value_description=="percent of population eating industrially produced vehicle"),
         "data_choice_notes"] = "Only one source."

subset_data['Sudan'] = subset_data['Sudan'].append(
    sudan.loc[(sudan.value_description=="percent of population eating industrially produced vehicle")]
)

sudan.loc[(sudan.value_description=="percent of population eating industrially produced vehicle")]

In [ ]:
mult_estimates[(mult_estimates.location_name=="Sudan") 
               & (mult_estimates.vehicle.isin(vehicles)) 
               & (mult_estimates.nutrient.isin(nutrients))]

## Madagascar

In [ ]:
mada = check_one_country("Madagascar")

mada[(mada.vehicle.isin(vehicles))].groupby(['vehicle','value_description']).mean()

In [ ]:
mult_estimates[(mult_estimates.location_name=="Madagascar") 
               & (mult_estimates.vehicle.isin(vehicles)) 
               & (mult_estimates.nutrient.isin(nutrients))]

## pop-weight subnationals

In [ ]:
#these are the subnats we have to weight

checkout = pd.concat(list(subset_data.values()))
checkout.loc[(checkout.subnational_name.notna()) & (checkout.subnational_name!='na'),
         ['location_name','urbanicity','subnational_name','source_link']].drop_duplicates()

## Check for missingness

In [ ]:
all_data = pd.concat(list(subset_data.values()))

In [ ]:
all_data.data_choice_notes.unique()

In [ ]:
all_data.loc[~(all_data.nutrient.isin(['vitamin a','iron','zinc','folic acid'])),'nutrient'] = 'na'

In [ ]:
all_data[rcols + ['value_mean','value_025_percentile','value_975_percentile','sub_population']]

In [ ]:
check = target.merge(all_data[rcols + ['value_mean']], on = rcols, how = 'left')

In [ ]:
check

In [ ]:
need_reg = check.loc[check.value_mean.isna(),['value_description','vehicle','location_name','nutrient']]

In [ ]:
need_reg = need_reg.merge(reg_estimates, on = ['value_description','vehicle','location_name','nutrient'], how = 'left')

In [ ]:
need_reg

In [ ]:
all_data = all_data.append(need_reg)

In [ ]:
all_data.value_mean = all_data.value_mean.astype(float)

In [ ]:
fort = all_data[(all_data.value_description=="percent of population eating fortified vehicle")]
other = all_data[(all_data.value_description!="percent of population eating fortified vehicle")]

In [ ]:
fort = pd.pivot_table(fort, index=['location_name','vehicle'],values = 'value_mean', columns = 'value_description')

In [ ]:
other = pd.pivot_table(other, index=['location_name','vehicle'],values = 'value_mean', columns = 'value_description')

In [ ]:
validate = fort.reset_index().merge(other.reset_index(), on = ['location_name','vehicle'], how = 'outer').set_index(['location_name','vehicle'])

In [ ]:
validate.columns = [i.replace(" ","_") for i in validate.columns]

In [ ]:
validate

In [ ]:
validate.loc[(validate.percent_of_population_eating_fortified_vehicle > validate.percent_of_population_eating_industrially_produced_vehicle) | (validate.percent_of_population_eating_industrially_produced_vehicle > validate.percent_of_population_eating_vehicle)]

In [ ]:
check = target.merge(all_data[rcols + ['value_mean']], on = rcols, how = 'left')

In [ ]:
assert(len(check[check.value_mean.isna()])==0), "there are target loc/vehicle/val/nutrient combos youre missing"

In [ ]:
rcols

In [ ]:
output = all_data[rcols + ['value_mean','value_025_percentile', 'value_975_percentile','sub_population','estimation_status','source_citation','source_link','data_choice_notes']].sort_values(rcols).set_index(rcols)

In [ ]:
##impute all missing CIs

# clean value_mean
output.loc[output.value_mean=='na','value_mean'] = np.nan
output.value_mean = output.value_mean.astype(float)

# clean 2.5th %ile
output.loc[output.value_025_percentile=='na','value_025_percentile'] = np.nan
output.value_025_percentile = output.value_025_percentile.astype(float)

# clean 97.5th %ile
output.loc[output.value_975_percentile=='na','value_975_percentile'] = np.nan
output.value_975_percentile = output.value_975_percentile.astype(float)

In [ ]:
output.loc[(output.value_mean > output.value_975_percentile),'value_975_percentile'] = np.nan

In [ ]:
output.loc[(output.value_mean < output.value_025_percentile)]

In [ ]:
output.loc[(output.value_mean == output.value_025_percentile)]

In [ ]:
output['scale_over_mean'] = (output.value_975_percentile - output.value_025_percentile) / output.value_mean

In [ ]:
output = output.reset_index()

In [ ]:
r = output.loc[(output.scale_over_mean!=np.inf),['vehicle','scale_over_mean']]
# .groupby('vehicle').mean().dropna().rename(columns={'scale_over_mean':'r'}).reset_index()

In [ ]:
r_mean = r.scale_over_mean.mean()

In [ ]:
r.loc[r.vehicle=="maize flour",'scale_over_mean'] = r_mean

In [ ]:
r = r.groupby('vehicle').mean().dropna().rename(columns={'scale_over_mean':'r'}).reset_index()

In [ ]:
r

In [ ]:
break

In [ ]:
# add uncertainty
output = output.merge(r, on = 'vehicle', how = 'outer')

In [ ]:
output.loc[(output.estimation_status=="regression"),'r'] = output.r * 2

In [ ]:
output['lower'] = np.clip(output.value_mean - (output.r * output.value_mean)/2, 0, 100)
output['upper'] = np.clip(output.value_mean + (output.r * output.value_mean)/2, 0, 100)

In [ ]:
output

In [ ]:
output.loc[(output.value_mean < output.value_025_percentile) | (output.value_025_percentile.isna()),'CI_source'] = "modeling"
output.loc[(output.value_mean > output.value_975_percentile) | (output.value_975_percentile.isna()),'CI_source'] = "modeling"

output.loc[output.CI_source.isna(),'CI_source'] = 'extraction'

output.loc[(output.value_mean < output.value_025_percentile) | (output.value_025_percentile.isna()),'value_025_percentile'] = output.loc[(output.value_mean < output.value_025_percentile) | (output.value_025_percentile.isna())].lower
output.loc[(output.value_mean > output.value_975_percentile) | (output.value_975_percentile.isna()),'value_975_percentile'] = output.loc[(output.value_mean > output.value_975_percentile) | (output.value_975_percentile.isna())].upper

output = output.drop(columns=['r','lower','upper','scale_over_mean']).set_index(rcols)

output.loc[output.estimation_status.isna(),'estimation_status'] = 'na'

In [ ]:
output

In [ ]:
output = output.reset_index()

In [ ]:
output.loc[(output.location_name=="Vietnam"),'location_name'] = "Viet Nam"

In [ ]:
sort_helper = {
    'percent of population eating fortified vehicle': 'C',
 'percent of population eating industrially produced vehicle': 'B',
 'percent of population eating vehicle': 'A'}

In [ ]:
output['sort_helper'] = output.value_description.map(sort_helper)

In [ ]:
output = output.sort_values(['location_name','vehicle','sort_helper','value_description','nutrient']).drop(columns='sort_helper')

In [ ]:
save_path_tmp = '/ihme/homes/beatrixh/repos/scratch/tier5_coverage_data_03_29_2021.csv'
output.to_csv(save_path_tmp, index = False)

In [ ]:
save_path = '/ihme/homes/beatrixh/vivarium_research_lsff/data_prep/outputs/tier5_coverage_data_03_29_2021.csv'
output.to_csv(save_path, index = False)